In [1]:
import os
import pathlib
import pickle

import anndata
import torch
import numpy as np
import scanpy as sc
import scarches as sca
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

from scarches.dataset.trvae.data_handling import remove_sparsity
# from lataq.models import EMBEDCVAE
# from lataq.exp_dict import EXPERIMENT_INFO

sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=500)
plt.rcParams['figure.figsize'] = (5, 5)
torch.set_printoptions(precision=3, sci_mode=False, edgeitems=7)

%load_ext autoreload
%autoreload 2

In [2]:
home_dir = os.path.expanduser("~")

In [3]:
HLCA_SCARCHES_TRAINED_MODEL = os.path.join(home_dir, "io/lataq_repr/scarches_models/HLCA_reference_model/")
HLCA_SCARCHES_GENE_ORDER = os.path.join(home_dir, "repositories/mapping_data_to_the_HLCA/supporting_files/HLCA_scarches_gene_order.csv")


TRAIN_DATA = os.path.join(home_dir, "data/HLCA/hlca_counts.h5ad")
SCARCHES_TRAIN_DATA = os.path.join(home_dir, "data/HLCA/HLCA_v1_scANVI_input.h5ad")
# QUERY_DATA = os.path.join(home_dir, "data/HLCA/extension_data_pooled_raw_2000genes.h5ad")
QUERY_DATA = (
    '/storage/groups/ml01/workspace/'
    'hlca_lisa.sikkema_malte.luecken/'
    'HLCA_reproducibility/data/HLCA_extended/'
    'extension_datasets/raw/Meyer/Meyer_2021_raw.h5ad'
)
HLCA_LABEL_RELATIONS = os.path.join(home_dir, "repositories/HLCA_reproducibility/supporting_files/metadata_harmonization/HLCA_cell_type_reference_mapping_20211103.csv")

In [4]:
# TODO: Shouldn;t we use something else?
condition_key = 'subject_ID'
cell_type_key = ['ann_finest_level']

surgery_epochs = 500
early_stopping_kwargs = {
    "early_stopping_metric": "val_unweighted_loss",
    "threshold": 0,
    "patience": 20,
    "reduce_lr": True,
    "lr_patience": 13,
    "lr_factor": 0.1,
}

In [5]:
PARAMS = {
    'EPOCHS': 50,                                      #TOTAL TRAINING EPOCHS
    'N_PRE_EPOCHS': 40,                                #EPOCHS OF PRETRAINING WITHOUT LANDMARK LOSS
    #'DATA_DIR': '../../lataq_reproduce/data',          #DIRECTORY WHERE THE DATA IS STORED
    #'DATA': 'pancreas',                                #DATA USED FOR THE EXPERIMENT
    'EARLY_STOPPING_KWARGS': {                         #KWARGS FOR EARLY STOPPING
        "early_stopping_metric": "val_landmark_loss",  ####value used for early stopping
        "mode": "min",                                 ####choose if look for min or max
        "threshold": 0,
        "patience": 20,
        "reduce_lr": True,
        "lr_patience": 13,
        "lr_factor": 0.1,
    },
    'LABELED_LOSS_METRIC': 'dist',           
    'UNLABELED_LOSS_METRIC': 'dist',
    'LATENT_DIM': 25,
    'ALPHA_EPOCH_ANNEAL': 1e3,
    'CLUSTERING_RES': 2,
    'HIDDEN_LAYERS': 3,
    'ETA': 5,
}

## Helper functions

In [6]:
# Inspired by https://github.com/LungCellAtlas/mapping_data_to_the_HLCA/blob/main/scripts/data_import_and_cleaning.py
def format_query_respect_to_reference(ref_anndata, query_anndata, ref_col='index', query_col='index'):
    if ref_col == 'index':
            ref_genes = ref_anndata.var.index.values
    else:
        ref_genes = ref_anndata.var[ref_col].astype('str').values
    if query_col == 'index':
        query_genes = query_anndata.var.index.values
    else:
        query_genes = query_anndata.var[ref_col].astype('str').values
    genes_to_add = set(ref_genes).difference(query_genes)
    diff = len(genes_to_add)
    if diff > 0:
        print(f'Not all genes were recovered, filling in zeros for {diff} missing genes...')
        # Genes to pad with
        df_padding = pd.DataFrame(data=np.zeros((query_anndata.shape[0], len(genes_to_add))), index=query_anndata.obs_names, columns=genes_to_add)
        adata_padding = sc.AnnData(df_padding, dtype=np.float32)
        # Concatenate object
        query_anndata = anndata.concat([query_anndata, adata_padding], axis=1, join='outer', index_unique=None, merge='unique')
    # subset and order:
    results_anndata = query_anndata[:, ref_genes].copy()
    results_anndata.var.index = ref_anndata.var.index.values
    return results_anndata

In [7]:
#!/bin/env python

import scanpy as sc
import pandas as pd


from collections import Counter

import numpy as np
from sklearn.neighbors import KNeighborsTransformer

def weighted_knn_trainer(train_adata, train_adata_emb, label_key, n_neighbors=50):
    """Trains a weighted KNN classifier on ``train_adata``.
    Parameters
    ----------
    train_adata: :class:`~anndata.AnnData`
        Annotated dataset to be used to train KNN classifier with ``label_key`` as the target variable.
    train_adata_emb: str
        Name of the obsm layer to be used for calculation of neighbors. If set to "X", anndata.X will be
        used
    label_key: str
        Name of the column to be used as target variable (e.g. cell_type) in ``train_adata`` and ``query_adata``.
    n_neighbors: int
        Number of nearest neighbors in KNN classifier.
    """
    print(
        f"Weighted KNN with n_neighbors = {n_neighbors} ... ",
        end="",
    )
    k_neighbors_transformer = KNeighborsTransformer(
        n_neighbors=n_neighbors,
        mode="distance",
        algorithm="brute",
        metric="euclidean",
        n_jobs=-1,
    )
    if train_adata_emb == "X":
        train_emb = train_adata.X
    elif train_adata_emb in train_adata.obsm.keys():
        train_emb = train_adata.obsm[train_adata_emb]
    else:
        raise ValueError(
            "train_adata_emb should be set to either 'X' or the name of the obsm layer to be used!"
        )
    k_neighbors_transformer.fit(train_emb)
    return k_neighbors_transformer


def weighted_knn_transfer(
    query_adata,
    query_adata_emb,
    ref_adata_obs,
    label_keys,
    knn_model,
    threshold=1,
    pred_unknown=False,
    mode="package",
):
    """Annotates ``query_adata`` cells with an input trained weighted KNN classifier.
    Parameters
    ----------
    query_adata: :class:`~anndata.AnnData`
        Annotated dataset to be used to queryate KNN classifier. Embedding to be used
    query_adata_emb: str
        Name of the obsm layer to be used for label transfer. If set to "X",
        query_adata.X will be used
    ref_adata_obs: :class:`pd.DataFrame`
        obs of ref Anndata
    label_keys: str
        Names of the columns to be used as target variables (e.g. cell_type) in ``query_adata``.
    knn_model: :class:`~sklearn.neighbors._graph.KNeighborsTransformer`
        knn model trained on reference adata with weighted_knn_trainer function
    threshold: float
        Threshold of uncertainty used to annotating cells as "Unknown". cells with
        uncertainties higher than this value will be annotated as "Unknown".
        Set to 1 to keep all predictions. This enables one to later on play
        with thresholds.
    pred_unknown: bool
        ``False`` by default. Whether to annotate any cell as "unknown" or not.
        If `False`, ``threshold`` will not be used and each cell will be annotated
        with the label which is the most common in its ``n_neighbors`` nearest cells.
    mode: str
        Has to be one of "paper" or "package". If mode is set to "package",
        uncertainties will be 1 - P(pred_label), otherwise it will be 1 - P(true_label).
    """
    if not type(knn_model) == KNeighborsTransformer:
        raise ValueError(
            "knn_model should be of type sklearn.neighbors._graph.KNeighborsTransformer!"
        )

    if query_adata_emb == "X":
        query_emb = query_adata.X
    elif query_adata_emb in query_adata.obsm.keys():
        query_emb = query_adata.obsm[query_adata_emb]
    else:
        raise ValueError(
            "query_adata_emb should be set to either 'X' or the name of the obsm layer to be used!"
        )
    top_k_distances, top_k_indices = k_neighbors_transformer.kneighbors(X=query_emb)

    stds = np.std(top_k_distances, axis=1)
    stds = (2.0 / stds) ** 2
    stds = stds.reshape(-1, 1)

    top_k_distances_tilda = np.exp(-np.true_divide(top_k_distances, stds))

    weights = top_k_distances_tilda / np.sum(
        top_k_distances_tilda, axis=1, keepdims=True
    )
    cols = ref_adata_obs.columns[ref_adata_obs.columns.str.startswith(label_keys)]
    uncertainties = pd.DataFrame(columns=cols, index=query_adata.obs_names)
    pred_labels = pd.DataFrame(columns=cols, index=query_adata.obs_names)
    for i in range(len(weights)):
        for j in cols:
            y_train_labels = ref_adata_obs[j].values
            unique_labels = np.unique(y_train_labels[top_k_indices[i]])
            best_label, best_prob = None, 0.0
            for candidate_label in unique_labels:
                candidate_prob = weights[
                    i, y_train_labels[top_k_indices[i]] == candidate_label
                ].sum()
                if best_prob < candidate_prob:
                    best_prob = candidate_prob
                    best_label = candidate_label

            if pred_unknown:
                if best_prob >= threshold:
                    pred_label = best_label
                else:
                    pred_label = "Unknown"
            else:
                pred_label = best_label

            if mode == "package":
                uncertainties.iloc[i][j] = (max(1 - best_prob, 0))

            else:
                raise Exception("Inquery Mode!")

            pred_labels.iloc[i][j] = (pred_label)

    print("finished!")

    return pred_labels, uncertainties

In [10]:
reference_adata = sc.read(SCARCHES_TRAIN_DATA)
reference_adata

AnnData object with n_obs × n_vars = 587218 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_ne

In [11]:
reference_emb_adata = sc.read(os.path.join(home_dir, "data/lataq_repr/scarches_models/HLCA/HLCA_emb_and_metadata.h5ad"))
reference_emb_adata

AnnData object with n_obs × n_vars = 584884 × 30
    obs: 'sample', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'BMI', 'condition', 'subject_type', 'sample_type', 'single_cell_platform', "3'_or_5'", 'sequencing_platform', 'cell_ranger_version', 'fresh_or_frozen', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_highest_res', 'n_genes', 'log10_total_counts', 'mito_frac', 'ribo_frac', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'scanvi_label', 'leiden_1', 'leiden_2', 'leiden_3', 'anatomical_region_ccf_score', 'entropy_study_leiden_3', 'entropy_dataset_leiden_3', 'entropy_subject_ID_leiden_3', 'entropy_original_ann_level_1_leiden_3', 'entropy_original_ann_level_2_clean_leiden_3', 'entropy_original_ann_level_3_clean_leiden_3', 'entropy_original_ann_level_4

In [12]:
gene_order_df = pd.read_csv(HLCA_SCARCHES_GENE_ORDER)
gene_order_df

,gene_id,gene_symbol
0,ENSG00000000938,FGR
1,ENSG00000000971,CFH
2,ENSG00000002587,HS3ST1
3,ENSG00000002933,TMEM176A
4,ENSG00000003436,TFPI
...,...,...
1995,ENSG00000280721,AC133644.2
1996,ENSG00000281103,TRG-AS1
1997,ENSG00000282122,CH17-262H11.1
1998,ENSG00000282988,RP1-34B20.21


In [13]:
query_adata = sc.read(QUERY_DATA)
query_adata

AnnData object with n_obs × n_vars = 128628 × 33694
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'ann_level_1_clean', 'ann_level_2_clean', 'ann_level_3_clean', 'ann_level_4_clean', 'ann_level_5_clean', 'subject_ID', 'subject_ID_as_published', 'Processing_site', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'dataset'
    var: 'gene_sym

In [14]:
query_adata = query_adata[:, gene_order_df.gene_id]
query_adata.obs["scanvi_label"] = "unlabeled"
query_adata

/home/icb/amirali.moinfar/miniconda3/envs/hlca/lib/python3.7/site-packages/ipykernel_launcher.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  


AnnData object with n_obs × n_vars = 128628 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'ann_level_1_clean', 'ann_level_2_clean', 'ann_level_3_clean', 'ann_level_4_clean', 'ann_level_5_clean', 'subject_ID', 'subject_ID_as_published', 'Processing_site', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'dataset', 'scanvi_label'
   

In [ ]:
# query_adata

In [ ]:
# query_adata = format_query_respect_to_reference(reference_adata, query_adata, ref_col='index', query_col='index')
# query_adata.obs["scanvi_label"] = "unlabeled"
# query_adata

In [ ]:
# sca_model = sca.models.SCANVI.load(HLCA_SCARCHES_TRAINED_MODEL, reference_adata)

In [ ]:
# data_latent = sca_model.get_latent_representation(
#     reference_adata,
# )
# adata_latent = sc.AnnData(data_latent)
# adata_latent.obs = reference_adata.obs.copy()

### Transfer model to CPU

In [ ]:
# TODO
# lataq_model.model.cpu()
# lataq_model.save(HLCA_TRAINED_MODEL + '.cpu')
# lataq_model = EMBEDCVAE.load(
#     HLCA_TRAINED_MODEL + '.cpu',
#     reference_adata,
# )

In [ ]:
# lataq_model.get_latent(
#     reference_adata.X.A.astype('float32'), 
#     reference_adata.obs[condition_key].values,
#     mean=True,
# )

## Perform surgery on reference model and train on query dataset

In [15]:
study = "Meyer_2021"
ref_path = HLCA_SCARCHES_TRAINED_MODEL

In [16]:
query_subadata = query_adata[query_adata.obs.study == study,:].copy()
query_subadata

AnnData object with n_obs × n_vars = 128628 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'ann_level_1_clean', 'ann_level_2_clean', 'ann_level_3_clean', 'ann_level_4_clean', 'ann_level_5_clean', 'subject_ID', 'subject_ID_as_published', 'Processing_site', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'dataset', 'scanvi_label'
   

In [17]:
%%time

surgery_epochs = 500
early_stopping_kwargs_surgery = {
    "early_stopping_metric": "elbo",
    "save_best_state_metric": "elbo",
    "on": "full_dataset",
    "patience": 10,
    "threshold": 0.001,
    "reduce_lr_on_plateau": True,
    "lr_patience": 8,
    "lr_factor": 0.1,
}

model = sca.models.SCANVI.load_query_data(
    query_subadata,
    ref_path,
    freeze_dropout = True,
)

model.train(
    n_epochs_semisupervised=surgery_epochs,
    train_base_model=False,
    semisupervised_trainer_kwargs=dict(
        metrics_to_monitor=["accuracy", "elbo"], 
        weight_decay=0,
        early_stopping_kwargs=early_stopping_kwargs_surgery
    ),
    frequency=1
)

INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']     
INFO     Successfully registered anndata object containing 128628 cells, 2000 vars, 16       
         batches, 29 labels, and 0 proteins. Also registered 0 extra categorical covariates  
         and 0 extra continuous covariates.                                                  
INFO     Training Unsupervised Trainer for 62 epochs.                                        
INFO     Training SemiSupervised Trainer for 500 epochs.                                     
INFO     KL warmup for 400 epochs                                                            
Training...:   5%|██████████▍                                                                                                                             

In [22]:
# latent_reference = model.get_latent_representation(
#     reference_adata,
# )
# latent_reference_adata = sc.AnnData(latent_reference)
# latent_reference_adata.obs = reference_adata.obs.copy()
# latent_reference_adata

AnnData object with n_obs × n_vars = 587218 × 30
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new'

In [24]:
reference_emb_adata

AnnData object with n_obs × n_vars = 584884 × 30
    obs: 'sample', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'BMI', 'condition', 'subject_type', 'sample_type', 'single_cell_platform', "3'_or_5'", 'sequencing_platform', 'cell_ranger_version', 'fresh_or_frozen', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_highest_res', 'n_genes', 'log10_total_counts', 'mito_frac', 'ribo_frac', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'scanvi_label', 'leiden_1', 'leiden_2', 'leiden_3', 'anatomical_region_ccf_score', 'entropy_study_leiden_3', 'entropy_dataset_leiden_3', 'entropy_subject_ID_leiden_3', 'entropy_original_ann_level_1_leiden_3', 'entropy_original_ann_level_2_clean_leiden_3', 'entropy_original_ann_level_3_clean_leiden_3', 'entropy_original_ann_level_4

In [25]:
latent_subdata = model.get_latent_representation(
    query_subadata,
)
latent_subadata = sc.AnnData(latent_subdata)
latent_subadata.obs = query_subadata.obs.copy()
latent_subadata

AnnData object with n_obs × n_vars = 128628 × 30
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new', 'ann_level_1_clean', 'ann_level_2_clean', 'ann_level_3_clean', 'ann_level_4_clean', 'ann_level_5_clean', 'subject_ID', 'subject_ID_as_published', 'Processing_site', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'dataset', 'scanvi_label', '_sc

In [26]:
def get_celltype_mappings(dataset_name, csv_file_path=HLCA_LABEL_RELATIONS):
    LEVELS = ['Level_1', 'Level_2', 'Level_3', 'Level_4', 'Level_5']
    main_mapping = pd.read_csv(csv_file_path, skiprows=1)
    main_mapping = main_mapping.replace(">", np.nan)
    main_mapping["ann_finest_level"] = main_mapping[LEVELS[-1]]
    for level in LEVELS[::-1]:
        main_mapping["ann_finest_level"] = main_mapping["ann_finest_level"].combine_first(main_mapping[level])
    main_mapping = main_mapping.fillna('-')
    main_mapping = main_mapping.replace("|", np.nan)
    for level in LEVELS:
        main_mapping[level] = main_mapping[level].ffill()
    main_mapping = main_mapping.replace("-", np.nan)
    final_mapping = main_mapping[[dataset_name, 'ann_finest_level']].dropna(subset=[dataset_name])
    return final_mapping

In [27]:
# Note: Got from HLCA notebbok reuslts

manual_to_fine_grained = {'NK cells': ['NK cells'],
 'Neuroendocrine': ['Neuroendocrine'],
 'CD8 T cells': ['CD8 T cells'],
 'Suprabasal': ['Suprabasal'],
 'Alveolar fibroblasts': ['Alveolar fibroblasts'],
 'Plasma cells': ['Plasma cells'],
 'Mast cells': ['Mast cells'],
 'Alveolar macrophages': ['Alveolar macrophages'],
 'SMG mucous': ['SMG mucous'],
 'AT1': ['AT1'],
 'Classical monocytes': ['Classical monocytes'],
 'Mesothelium': ['Mesothelium'],
 'DC2': ['DC2'],
 'Non-classical monocytes': ['Non-classical monocytes'],
 'DC1': ['DC1'],
 'Deuterosomal': ['Deuterosomal'],
 'AT2 proliferating': ['AT2 proliferating'],
 'Pericytes': ['Pericytes'],
 'EC venous pulmonary': ['EC venous pulmonary'],
 'AT2': ['AT2'],
 'EC venous systemic': ['EC venous systemic'],
 'Plasmacytoid DCs': ['Plasmacytoid DCs'],
 'CD4 T cells': ['CD4 T cells'],
 'SMG serous': ['SMG serous (nasal)', 'SMG serous (bronchial)'],
 'Secretory': ['Club (non-nasal)',
  'Club (nasal)',
  'Goblet (nasal)',
  'Goblet (bronchial)',
  'Goblet (subsegmental)',
  'Transitional Club-AT2'],
 'Basal': ['Basal resting', 'Suprabasal'],
 'Lymphatic EC': ['Lymphatic EC mature',
  'Lymphatic EC proliferating',
  'Lymphatic EC differentiating'],
 'Goblet': ['Goblet (nasal)', 'Goblet (bronchial)', 'Goblet (subsegmental)'],
 'Fibroblasts': ['Peribronchial fibroblasts',
  'Adventitial fibroblasts',
  'Alveolar fibroblasts',
  'Pericytes',
  'Subpleural fibroblasts'],
 'Club': ['Club (non-nasal)', 'Club (nasal)'],
 'Rare': ['Ionocyte', 'Tuft', 'Neuroendocrine'],
 'Interstitial macrophages': ['Monocyte-derived Mφ',
  'Interstitial Mφ perivascular'],
 'Multiciliated': ['Multiciliated (nasal)', 'Multiciliated (non-nasal)'],
 'EC capillary': ['EC aerocyte capillary', 'EC general capillary'],
 'Mature B cells': ['B cells'],
 'NK_ITGAD+': ['NK cells'],
 'CD4T cells TRM': ['CD4 T cells'],
 'Arterial Pulmonary': ['EC arterial'],
 'Arterial Systemic': ['EC arterial'],
 'CD4 T cells MAIT': ['CD4 T cells'],
 'CD8 T cells TRM': ['CD8 T cells'],
 'CD4 T cells naive': ['CD4 T cells'],
 'Airway smooth muscle': ['Smooth muscle'],
 'NK_XCL1+': ['NK cells'],
 'CD8 T cells GZMK+': ['CD8 T cells'],
 'Plasmablasts': ['Plasma cells'],
 'Vascular smooth muscle': ['Smooth muscle'],
 'Naive B cells': ['B cells'],
 'CD8 T cells ME': ['CD8 T cells'],
 'Erythrocytes': ['New'],
 'Basal proliferating': ['Basal resting', 'Suprabasal'],
 'Schwann myelinating': ['New'],
 'Gamma-delta T cells': ['New'],
 'DC activated': ['Migratory DCs'],
 'Megakaryocytes': ['New'],
 'NKT cells': ['New'],
 'Chondrocytes': ['New'],
 'Regulatory T cells': ['New'],
 'ILCs': ['New'],
 'Mφ proliferating': ['Alveolar Mφ proliferating',
  'Interstitial Mφ perivascular',
  'Monocyte-derived Mφ',
  'Alveolar macrophages'],
 'Schwann nonmyelinating': ['New']}

# Note: for compatibility with Carlo
manual_to_fine_grained['Club'] += 'Transitional Club-AT2'
# My division
manual_to_fine_grained['EC aerocyte capillary'] =  ['EC aerocyte capillary']
manual_to_fine_grained['EC general capillary'] =  ['EC general capillary']


manual_to_fig_label = {'NK cells': 'NK cells',
 'Neuroendocrine': 'Neuroendocrine',
 'CD8 T cells': 'CD8 T cells',
 'Suprabasal': 'Suprabasal',
 'Alveolar fibroblasts': 'Alveolar fibroblasts',
 'Plasma cells': 'Plasma cells',
 'Mast cells': 'Mast cells',
 'Alveolar macrophages': 'Alveolar macrophages',
 'SMG mucous': 'SMG mucous',
 'AT1': 'AT1',
 'Classical monocytes': 'Classical monocytes',
 'Mesothelium': 'Mesothelium',
 'DC2': 'DC2',
 'Non-classical monocytes': 'Non-classical monocytes',
 'DC1': 'DC1',
 'Deuterosomal': 'Deuterosomal',
 'AT2 proliferating': 'AT2 proliferating',
 'Pericytes': 'Pericytes',
 'EC venous pulmonary': 'EC venous pulmonary',
 'AT2': 'AT2',
 'EC venous systemic': 'EC venous systemic',
 'Plasmacytoid DCs': 'Plasmacytoid DCs',
 'CD4 T cells': 'CD4 T cells',
 'Mature B cells': 'B cells',
 'NK_ITGAD+': 'NK cells',
 'CD4T cells TRM': 'CD4 T cells',
 'Arterial Pulmonary': 'EC arterial',
 'Arterial Systemic': 'EC arterial',
 'CD4 T cells MAIT': 'CD4 T cells',
 'CD8 T cells TRM': 'CD8 T cells',
 'CD4 T cells naive': 'CD4 T cells',
 'Airway smooth muscle': 'Smooth muscle',
 'NK_XCL1+': 'NK cells',
 'CD8 T cells GZMK+': 'CD8 T cells',
 'Plasmablasts': 'Plasma cells',
 'Vascular smooth muscle': 'Smooth muscle',
 'Naive B cells': 'B cells',
 'CD8 T cells ME': 'CD8 T cells',
 'Basal proliferating': 'Basal',
 'Mφ proliferating': 'Macrophages',
 'SMG serous': 'SMG serous',
 'Secretory': 'Secretory',
 'Basal': 'Basal',
 'Lymphatic EC': 'Lymphatic EC',
 'Goblet': 'Goblet',
 'Fibroblasts': 'Fibroblasts',
 'Club': 'Club',
 'Rare': 'Rare',
 'Interstitial macrophages': 'Interstitial macrophages',
 'Multiciliated': 'Multiciliated',
 'EC aerocyte capillary': 'EC aerocyte capillary',
 'EC general capillary': 'EC general capillary',
 'Erythrocytes': 'Erythrocytes',
 'Schwann myelinating': 'Schwann myelinating',
 'Gamma-delta T cells': 'Gamma-delta T cells',
 'DC activated': 'Migratory DCs',
 'Megakaryocytes': 'Megakaryocytes',
 'NKT cells': 'NKT cells',
 'Chondrocytes': 'Chondrocytes',
 'Regulatory T cells': 'Regulatory T cells',
 'ILCs': 'ILCs',
 'Schwann nonmyelinating': 'Schwann nonmyelinating'}

ct_ordered_q = ['Basal',
 'Suprabasal',
 'Deuterosomal',
 'Multiciliated',
 'Club',
 'Secretory',
 'Goblet',
 'Rare',
 'Neuroendocrine',
 'SMG serous',
 'SMG mucous',
 'AT1',
 'AT2',
 'AT2 proliferating',
 'EC arterial',
 'EC aerocyte capillary',
 'EC general capillary',
 'EC venous systemic',
 'EC venous pulmonary',
 'Lymphatic EC',
 'Fibroblasts',
 'Alveolar fibroblasts',
 'Pericytes',
 'Smooth muscle',
 'Mesothelium',
 'B cells',
 'Plasma cells',
 'CD4 T cells',
 'CD8 T cells',
 'NK cells',
 'DC1',
 'DC2',
 'Migratory DCs',
 'Plasmacytoid DCs',
 'Alveolar macrophages',
 'Macrophages',
 'Interstitial macrophages',
 'Classical monocytes',
 'Non-classical monocytes',
 'Mast cells',
 'Chondrocytes',
 'Erythrocytes',
 'Gamma-delta T cells',
 'ILCs',
 'Megakaryocytes',
 'NKT cells',
 'Regulatory T cells',
 'Schwann myelinating',
 'Schwann nonmyelinating']

In [28]:
acceptable_predictions_df = pd.DataFrame([(k, v) for k, vl in manual_to_fine_grained.items() for v in vl], columns=['manual_ann', 'pred'])
acceptable_predictions_df

,manual_ann,pred
0,NK cells,NK cells
1,Neuroendocrine,Neuroendocrine
2,CD8 T cells,CD8 T cells
3,Suprabasal,Suprabasal
4,Alveolar fibroblasts,Alveolar fibroblasts
...,...,...
104,Mφ proliferating,Monocyte-derived Mφ
105,Mφ proliferating,Alveolar macrophages
106,Schwann nonmyelinating,New
107,EC aerocyte capillary,EC aerocyte capillary


In [29]:
%%time

# run k-neighbors transformer
k_neighbors_transformer = weighted_knn_trainer(
    train_adata=reference_emb_adata,
    train_adata_emb="X", # location of our joint embedding
    label_key="ann_finest_level",
    n_neighbors=50,
    )    
# perform label transfer
labels, uncert = weighted_knn_transfer(
    query_adata=latent_subadata,
    query_adata_emb="X", # location of our joint embedding
    label_keys="ann_finest_level",
    knn_model=k_neighbors_transformer,
    ref_adata_obs = reference_emb_adata.obs
    )

Weighted KNN with n_neighbors = 50 ... finished!
CPU times: user 54min 57s, sys: 19min 5s, total: 1h 14min 3s
Wall time: 57min 23s


In [42]:
cts_ordered = pd.read_csv(
    os.path.join(home_dir, "repositories/HLCA_reproducibility/supporting_files/celltype_structure_and_colors/manual_anns_and_leveled_anns_ordered.csv"), index_col=0
)

In [43]:
labels, uncert

(['Correct', 'Incorrect', 'Unknown'],
                                    ann_finest_level
 AACTTTCAGATGTGGC-WSSS8123924               0.019974
 ACCTTTAGTGAGTGAC-WSSS8123924               0.159995
 ACGATACTCCGCTGTT-WSSS8123924               0.100003
 ACTGAACGTAGTGAAT-WSSS8123924               0.139987
 AGAGTGGTCAACACTG-WSSS8123924                    0.0
 ...                                             ...
 TTTGTCAGTTATCCGA-HCATisStab7732262         0.059996
 TTTGTCAGTTCAGTAC-HCATisStab7732262              0.0
 TTTGTCATCACCGGGT-HCATisStab7732262              0.0
 TTTGTCATCGCGTTTC-HCATisStab7732262              0.0
 TTTGTCATCTAACGGT-HCATisStab7732262         0.259961
 
 [128628 rows x 1 columns])

In [32]:
def get_celltype_mappings(dataset_name, csv_file_path=HLCA_LABEL_RELATIONS):
    LEVELS = ['Level_1', 'Level_2', 'Level_3', 'Level_4', 'Level_5']
    main_mapping = pd.read_csv(csv_file_path, skiprows=1)
    main_mapping = main_mapping.replace(">", np.nan)
    main_mapping["ann_finest_level"] = main_mapping[LEVELS[-1]]
    for level in LEVELS[::-1]:
        main_mapping["ann_finest_level"] = main_mapping["ann_finest_level"].combine_first(main_mapping[level])
    main_mapping = main_mapping.fillna('-')
    main_mapping = main_mapping.replace("|", np.nan)
    for level in LEVELS:
        main_mapping[level] = main_mapping[level].ffill()
    main_mapping = main_mapping.replace("-", np.nan)
    final_mapping = main_mapping[[dataset_name, 'ann_finest_level']].dropna(subset=[dataset_name])
    return final_mapping

In [33]:
mapping_df = get_celltype_mappings('Meyer_2021')
mapping_df

,Meyer_2021,ann_finest_level
2,Basal,Basal
6,Suprabasal,Suprabasal
8,Dividing_Basal,Basal proliferating
16,Deuterosomal,Deuterosomal
19,Ciliated,Multiciliated
...,...,...
257,Mast_cell,Mast cells
264,Megakaryocyte,Megakaryocytes
265,Erythrocyte,Erythrocytes
269,Schwann_nonmyelinating,Schwann nonmyelinating


In [41]:
label_transfer_df_2 = pd.DataFrame({
    'original_pred': labels["ann_finest_level"].values,
    'prob': uncert["ann_finest_level"].values,
    'manual_ann': query_subadata.obs['original_celltype_ann'].values
})

label_transfer_df_2 = (
    label_transfer_df_2
    .merge(mapping_df.rename({'Meyer_2021': 'manual_ann'}, axis=1), 
           on='manual_ann', 
           how='left')
    .drop(columns='manual_ann')
    .rename({'ann_finest_level': 'manual_ann'}, axis=1)
)

label_transfer_df_2

TypeError: list indices must be integers or slices, not str

In [69]:
label_transfer_df_2

,original_pred,prob,manual_ann,pred,status,manual_ann_fig
0,Interstitial Mφ perivascular,0.019974,Interstitial macrophages,Interstitial Mφ perivascular,Correct,Interstitial macrophages
1,Adventitial fibroblasts,0.159995,Fibroblasts,Adventitial fibroblasts,Unknown,Fibroblasts
2,Lymphatic EC differentiating,0.100003,Lymphatic EC,Lymphatic EC differentiating,Unknown,Lymphatic EC
3,Adventitial fibroblasts,0.139987,Fibroblasts,Adventitial fibroblasts,Unknown,Fibroblasts
4,Adventitial fibroblasts,0.0,Fibroblasts,Adventitial fibroblasts,Correct,Fibroblasts
...,...,...,...,...,...,...
128623,CD4 T cells,0.059996,CD4 T cells,CD4 T cells,Correct,CD4 T cells
128624,Mast cells,0.0,Mast cells,Mast cells,Correct,Mast cells
128625,NK cells,0.0,NK cells,NK cells,Correct,NK cells
128626,CD8 T cells,0.0,Gamma-delta T cells,CD8 T cells,Incorrect,Gamma-delta T cells


In [84]:
UNCERTAINTY_THRESHOLD = 0.3
label_transfer_df_2['pred'] = np.where(
    label_transfer_df_2['prob'] < UNCERTAINTY_THRESHOLD,
    label_transfer_df_2['original_pred'].copy(),
    'Unknown'
)

label_transfer_df_2 = label_transfer_df_2.merge(acceptable_predictions_df.assign(status='Correct'), how='left').fillna({'status': 'Incorrect'})
label_transfer_df_2['status'] = np.where(label_transfer_df_2['pred'] == 'Unknown', 'Unknown', label_transfer_df_2['status'])
label_transfer_df_2

,original_pred,prob,manual_ann,pred,manual_ann_fig,status
0,Interstitial Mφ perivascular,0.019974,Interstitial macrophages,Interstitial Mφ perivascular,Interstitial macrophages,Correct
1,Adventitial fibroblasts,0.159995,Fibroblasts,Adventitial fibroblasts,Fibroblasts,Correct
2,Lymphatic EC differentiating,0.100003,Lymphatic EC,Lymphatic EC differentiating,Lymphatic EC,Correct
3,Adventitial fibroblasts,0.139987,Fibroblasts,Adventitial fibroblasts,Fibroblasts,Correct
4,Adventitial fibroblasts,0.0,Fibroblasts,Adventitial fibroblasts,Fibroblasts,Correct
...,...,...,...,...,...,...
128623,CD4 T cells,0.059996,CD4 T cells,CD4 T cells,CD4 T cells,Correct
128624,Mast cells,0.0,Mast cells,Mast cells,Mast cells,Correct
128625,NK cells,0.0,NK cells,NK cells,NK cells,Correct
128626,CD8 T cells,0.0,Gamma-delta T cells,CD8 T cells,Gamma-delta T cells,Incorrect


In [85]:
label_transfer_df_2['manual_ann_fig'] = label_transfer_df_2.manual_ann.map(manual_to_fig_label)

In [86]:
label_transfer_df_2.to_csv(os.path.expanduser("~/tmp/scarches_knn_uncert_results.csv"))

In [87]:
total_n_per_ct = label_transfer_df_2.manual_ann_fig.value_counts()
total_n_per_ct["Overall"] = label_transfer_df_2.shape[0]

perc_correct = (pd.crosstab(
    label_transfer_df_2.manual_ann_fig, label_transfer_df_2.status
)
#                 .loc[sorted(set(label_transfer_df_2.manual_ann.values)), :]
                .loc[ct_ordered_q, :]
               )
perc_correct.loc["Overall", :] = (
    label_transfer_df_2.status.value_counts() / total_n_per_ct["Overall"] * 100
)
perc_correct = perc_correct.div(perc_correct.sum(axis=1), axis="rows") * 100



with plt.rc_context(
    {
        "figure.figsize": (0.4 * len(perc_correct), 3),
        "axes.spines.right": False,
        "axes.spines.top": False,
    }
):
    fig, ax = plt.subplots()
    perc_correct.plot(kind="bar", stacked=True, ax=ax)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[::-1], labels[::-1], loc=(1.01, 0.60), frameon=False)

#     cts_no_underscore = [ct.replace("_", " ") for ct in cts_ordered_custom]
#     if incl_celln_in_label:
#         plt.xticks(
#             ticks=range(len(cts_ordered_q) + 1),
#             labels=[
#                 f"{ct_no_und} ({total_n_per_ct[ct]})"
#                 for ct_no_und, ct in zip(
#                     cts_no_underscore + ["Overall"], cts_ordered_custom #cts_ordered_q + ["Overall"]
#                 )
#             ],
#         )
#         plt.xlabel("Original label (n cells)")
#     else:
#         plt.xticks(
#             ticks=range(len(cts_ordered_custom)),
#             labels=[f"{ct_no_und}" for ct_no_und in cts_no_underscore],
#         )
#         plt.xlabel("Original label")
    ax.set_ylabel("% of cells")
    plt.grid(False)